In [122]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import nltk
import re
import spacy
from gensim import corpora, models
import gensim
from gensim.matutils import hellinger
from scipy.spatial.distance import cosine
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

In [3]:
nlp = spacy.load('en_core_web_sm')

In [57]:
df = pd.read_csv(r'C:\Users\trevo\NLP\data\ReducedDataset.csv')

In [58]:
df.head(5)

,Unnamed: 0,ALink,SName,SLink,Lyric,language
0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\nThis beautiful b...,en
1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\nOh how I needed you toda...",en
2,162905,/arch-enemy/,Despicable Heroes,/arch-enemy/despicable-heroes.html,"I spit in your face, preacers and leaders\nSpe...",en
3,281035,/the-maine/,Whoever She Is,/the-maine/whoever-she-is.html,I thought I had my girl but she ran away\nMy c...,en
4,253213,/a-ha/,Days On End,/a-ha/days-on-end.html,Do know why winter's such a cold and lonely pl...,en


In [59]:
df_en = df[df['language']=='en']

In [60]:
df_en.head(5)

,Unnamed: 0,ALink,SName,SLink,Lyric,language
0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\nThis beautiful b...,en
1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\nOh how I needed you toda...",en
2,162905,/arch-enemy/,Despicable Heroes,/arch-enemy/despicable-heroes.html,"I spit in your face, preacers and leaders\nSpe...",en
3,281035,/the-maine/,Whoever She Is,/the-maine/whoever-she-is.html,I thought I had my girl but she ran away\nMy c...,en
4,253213,/a-ha/,Days On End,/a-ha/days-on-end.html,Do know why winter's such a cold and lonely pl...,en


In [61]:
df_en.shape[0]

38363

In [133]:
print(df_en['Lyric'].iloc[0])

Come and I'll take you under
This beautiful bruise's colors
Everything fades in time, it's true
I wish that I had another
Stab at the undercover
Was it a change in mind for you

It's impossible
I can't let it out
You'll never know
Am I selling you out
Sit and watch your every mood

Your eyes still remind me of
Angels that hover above
Eyes that can change from blind to blue

It's impossible
I can't let it out
You'll never know
Am I selling you out
Sit and watch your every mood

Now that I've found my reward
I'd throw it away long before
I'd share a piece of mine with you

It's impossible
I can't let it out
You'll never know
Am I selling you out
Sit and watch your every mood


In [135]:
# Download the stopwords library
nltk.download('stopwords')

# Establish a word punctuation tokenizer
wpt = nltk.WordPunctTokenizer()

# Establish the English stop words
basic_stop_words = nltk.corpus.stopwords.words('english')

custom_stop_words = ['get', 'yeah', 's', 'ai', 'ca', 'like', 'nt', 'ta', 'oh', 'got', 'gonna','goin','na','I']

stop_words = basic_stop_words + custom_stop_words

def normalize_document(doc):
    # Lowercase and remove special characters and whitespaces
    doc = re.sub(r"[^a-zA-Z\s']", '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()

    # Tokenize document
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]
    # Re-create the document from filtered tokens
    doc = ' '.join(filtered_tokens)

        # Remove punctuation
    doc = re.sub(f"[{re.escape(string.punctuation)}]", '', doc)

    doc = re.sub(r"'\s*", "", doc)
    return doc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trevo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [136]:
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(df_en['Lyric'])

In [137]:
print(norm_corpus[0])

come  take beautiful bruise  colors everything fades time  true wish another stab undercover change mind  impossible  let  never know selling sit watch every mood eyes still remind angels hover eyes change blind blue  impossible  let  never know selling sit watch every mood  found reward  throw away long  share piece mine  impossible  let  never know selling sit watch every mood


In [139]:
doc = nlp(norm_corpus[0].item())

for token in doc:
    print(token.text, token.lemma_)

come come
   
take take
beautiful beautiful
bruise bruise
   
colors color
everything everything
fades fade
time time
   
true true
wish wish
another another
stab stab
undercover undercover
change change
mind mind
   
impossible impossible
   
let let
   
never never
know know
selling sell
sit sit
watch watch
every every
mood mood
eyes eye
still still
remind remind
angels angel
hover hover
eyes eye
change change
blind blind
blue blue
   
impossible impossible
   
let let
   
never never
know know
selling sell
sit sit
watch watch
every every
mood mood
   
found find
reward reward
   
throw throw
away away
long long
   
share share
piece piece
mine mine
   
impossible impossible
   
let let
   
never never
know know
selling sell
sit sit
watch watch
every every
mood mood


In [140]:
lemmatized_corpus = []

for text in norm_corpus[0:5000]:
    doc = nlp(text.item())  
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    lemmatized_corpus.append(lemmatized_text)

In [142]:
print(lemmatized_corpus[0])

come   take beautiful bruise   color everything fade time   true wish another stab undercover change mind   impossible   let   never know sell sit watch every mood eye still remind angel hover eye change blind blue   impossible   let   never know sell sit watch every mood   find reward   throw away long   share piece mine   impossible   let   never know sell sit watch every mood


In [130]:
df = pd.DataFrame(lemmatized_corpus, columns=['lyrics'])
df.to_csv('output_file.csv', index=False)

In [143]:
tokenized_corpus = [text.split() for text in lemmatized_corpus]

dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

In [144]:
top_10_most_frequent_words = sorted(dictionary.cfs.items(), key=lambda item: item[1], reverse=True)[:10]
print(top_10_most_frequent_words)
for token_id, frequency in top_10_most_frequent_words:
    word = dictionary[token_id]
    print(f"Word: {word}, Frequency: {frequency}")

[(17, 8632), (203, 8532), (246, 6465), (9, 5112), (210, 5103), (145, 4893), (69, 4804), (34, 4563), (106, 4470), (18, 4147)]
Word: know, Frequency: 8632
Word: love, Frequency: 8532
Word: go, Frequency: 6465
Word: come, Frequency: 5112
Word: say, Frequency: 5103
Word: make, Frequency: 4893
Word: see, Frequency: 4804
Word: time, Frequency: 4563
Word: one, Frequency: 4470
Word: let, Frequency: 4147


In [88]:
lda_model = gensim.models.LdaModel(corpus, num_topics=40, id2word=dictionary, passes=55, random_state = 42)

In [89]:
for topic_id in range(20):
    topic = lda_model.print_topic(topic_id)
    print(f"Topic {topic_id}: {topic}")

Topic 0: 0.113*"na" + 0.022*"mother" + 0.016*"." + 0.014*"fake" + 0.014*"house" + 0.013*"master" + 0.013*"gangster" + 0.011*"mouth" + 0.011*"fuckin" + 0.010*"p"
Topic 1: 0.132*"high" + 0.074*"low" + 0.060*"river" + 0.048*"bad" + 0.030*"ay" + 0.019*"blame" + 0.018*"drunk" + 0.017*"sad" + 0.016*"yea" + 0.016*"go"
Topic 2: 0.123*"lie" + 0.068*"money" + 0.041*"send" + 0.029*"circle" + 0.020*"ghost" + 0.020*"town" + 0.017*"thief" + 0.016*"bye" + 0.016*"rich" + 0.015*"poor"
Topic 3: 0.323*"love" + 0.022*"heart" + 0.017*"make" + 0.016*"give" + 0.014*"true" + 0.011*"know" + 0.010*"need" + 0.010*"world" + 0.010*"every" + 0.009*"find"
Topic 4: 0.167*"light" + 0.108*"shine" + 0.057*"sun" + 0.055*"year" + 0.046*"bright" + 0.026*"beautiful" + 0.026*"life" + 0.022*"white" + 0.020*"day" + 0.016*"may"
Topic 5: 0.068*"red" + 0.035*"bear" + 0.034*"king" + 0.031*"whoa" + 0.022*"jesus" + 0.021*"star" + 0.020*"son" + 0.020*"wine" + 0.016*"blue" + 0.016*"side"
Topic 6: 0.055*"know" + 0.031*"time" + 0.022*"w

In [90]:
word_counts = {}
num_topics = lda_model.num_topics

for topic_id in range(num_topics):
    topic_words = lda_model.show_topic(topic_id, topn=10) 

    for word, prob in topic_words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

word_count_topic_list = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])

word_count_topic_list = word_count_topic_list.sort_values(by='Count', ascending=False)

print(word_count_topic_list.head(20))

      Word  Count
35    know      7
62     see      6
32    make      5
19      go      4
60    time      3
71     say      3
74    take      3
48     day      3
46    life      3
33    give      3
91    fall      3
64   never      3
97    hear      2
242   long      2
101   back      2
128   stop      2
133  alone      2
154     we      2
239   come      2
39    find      2


In [91]:
song_topic_distribution = [lda_model[doc] for doc in corpus]

In [92]:
song_topic_distribution[496]

[(6, 0.30719507),
 (13, 0.061509743),
 (16, 0.016949473),
 (19, 0.29525393),
 (27, 0.11779578),
 (28, 0.03979514),
 (30, 0.112560265),
 (36, 0.04397052)]

In [93]:
print(df_en['Lyric'].iloc[496])

Oh

I push it down
I take a deep breath
Try to think a little less
He lays me down
It's pretty good sex
Out of context
Yeah, yeah

I been tryin' to move on
And it's obvious that I can't
It was my fault we're broken
But I can't let go of hopin'
So I leave my door wide open

All my friends keep telling me
I just need to fuck someone new
Whatever I, ever I do
I'm gonna, I'm gonna think of you
So if it seems like somebody took your spot
Well, that's just not true
Whatever I, ever I do
I'm gonna, I'm gonna think of you

I'ma think about you
I'ma think about you (You, you, you, you, you)
I'ma think about you, oh
(You, you, you, you, you)

Hit me up, ring me one time (Brr)
If you wanna hit it one time
If you wanna fight, fuck me outta spite
If you need to hate me, that's fine
If I gave you all of my time
Could you get over your pride?
Hit me up, ring me one time
If you need to hate me, that's fine

I been tryin' to move on
And it's obvious that I can't
It was my fault we're broken
But I can't

In [94]:
song_topic_distribution[497]

[(6, 0.355395),
 (10, 0.028450787),
 (17, 0.016861463),
 (18, 0.036024816),
 (24, 0.02317726),
 (28, 0.52918875)]

In [95]:
print(df_en['Lyric'].iloc[497])

I'm in a bunker now, smoking a cigarette
I'd take my helmet off but the war's not over yet
Slow my trembling hands, slow my wanting heart
Keep me calm 'til I find a way to start

It's always hard to find the courage to start dancing
But I am determined to take my chances
But peace can be hard to find in all this misunderstanding
I am determined to take my chances

I get hypnotized, I pretend I'm smart
I'm always looking out for my getaway car
Some days are bound to hurt but which ones we'll never know
Some days you catch a break, some days you don't

It's always hard to find the courage to keep on dancing
I am determined to take my chances
Peace can be hard to find in all this misunderstanding
But I am determined to take my chances

It's always hard to find the courage to keep on dancing
We are determined to take our chances


In [96]:
song_0_dist = song_topic_distribution[496]
song_1_dist = song_topic_distribution[497]

hellinger_distance = hellinger(song_0_dist, song_1_dist)

print(hellinger_distance)

0.7186995293020352


In [99]:
def calculate_hellinger_distance(song_dist_1, song_dist_2):
    return hellinger(song_dist_1, song_dist_2)

input_song_index = 496

input_song_dist = song_topic_distribution[input_song_index]

most_similar_song_index = None
min_distance = float('inf') 

for i, song_dist in enumerate(song_topic_distribution):
    if i != input_song_index:
        distance = calculate_hellinger_distance(input_song_dist, song_dist)
        print(f"Hellinger distance between input song and song {i}: {distance}")

        if distance < min_distance:
            min_distance = distance
            most_similar_song_index = i

print(f"\nMost similar song to the input song is song {most_similar_song_index} with Hellinger distance: {min_distance}")

Hellinger distance between input song and song 0: 0.41255669616787743
Hellinger distance between input song and song 1: 0.7111655333284856
Hellinger distance between input song and song 2: 0.9951407578426981
Hellinger distance between input song and song 3: 0.6523077481297272
Hellinger distance between input song and song 4: 0.8412852875753029
Hellinger distance between input song and song 5: 0.6263216232502951
Hellinger distance between input song and song 6: 0.87463132834307
Hellinger distance between input song and song 7: 0.6169215197357198
Hellinger distance between input song and song 8: 0.9044364068205958
Hellinger distance between input song and song 9: 0.9941190805929552
Hellinger distance between input song and song 10: 0.869841211797022
Hellinger distance between input song and song 11: 0.7281808814229505
Hellinger distance between input song and song 12: 0.5611192017370927
Hellinger distance between input song and song 13: 0.8408160308141556
Hellinger distance between input

In [101]:
song_topic_distribution[496]

[(6, 0.30719507),
 (13, 0.061509743),
 (16, 0.016949473),
 (19, 0.29525393),
 (27, 0.11779578),
 (28, 0.03979514),
 (30, 0.112560265),
 (36, 0.04397052)]

In [103]:
print(df_en['SName'].iloc[496])
print(df_en['Lyric'].iloc[496])

Think About You
Oh

I push it down
I take a deep breath
Try to think a little less
He lays me down
It's pretty good sex
Out of context
Yeah, yeah

I been tryin' to move on
And it's obvious that I can't
It was my fault we're broken
But I can't let go of hopin'
So I leave my door wide open

All my friends keep telling me
I just need to fuck someone new
Whatever I, ever I do
I'm gonna, I'm gonna think of you
So if it seems like somebody took your spot
Well, that's just not true
Whatever I, ever I do
I'm gonna, I'm gonna think of you

I'ma think about you
I'ma think about you (You, you, you, you, you)
I'ma think about you, oh
(You, you, you, you, you)

Hit me up, ring me one time (Brr)
If you wanna hit it one time
If you wanna fight, fuck me outta spite
If you need to hate me, that's fine
If I gave you all of my time
Could you get over your pride?
Hit me up, ring me one time
If you need to hate me, that's fine

I been tryin' to move on
And it's obvious that I can't
It was my fault we're br

In [102]:
song_topic_distribution[274]

[(6, 0.45076847),
 (8, 0.013714441),
 (13, 0.025359526),
 (16, 0.077330165),
 (19, 0.2620164),
 (27, 0.04079083),
 (30, 0.10587079)]

In [104]:
print(df_en['SName'].iloc[274])
print(df_en['Lyric'].iloc[274])

Damned If I do Ya (damned If I Don't)
I fought it for a long time now
While drowning in a river of denial
I washed up, fixed up, picked up
All my broken things

'Cause you left me
Police scene, chalk line
Tequila shots
In the dark scene of the crime
Suburban living with a feeling
That I'm giving up
Everything for you
(For you)

Oh, oh, oh
How was I supposed to know
That you were oh, oh, over me?
I think that I should go
(Go!)
Something's telling me to leave
But I won't
'Cause I'm damned if I do ya
Damned if I don't

It took a lot to take you home
One stupid call
And I end up alone
You made up, dressed up, messed up
Plans I set in stone

But you made me do it
And I dont like dancing in the alley
With a streetrat night life
Can't keep living with a feeling
That I'm giving up
Everything for you
(For you)

Oh, oh, oh
How was I supposed to know
That you were oh, oh, over me?
I think that I should go
(Go!)
Something's telling me to leave
But I won't
'Cause I'm damned if I do ya
Damned if I d

In [85]:
print(df_en['Lyric'].iloc[52])

Time flies by when the night is young
Daylight shines on an undisclosed location,
location
Bloodshot eyes looking for the sun
Paradise delivered and
we call it a vacation, vacation

You're painting me a dream that I
Wanna belong, yeah, wanna belong, yeah

Over the hills and far away
A million miles from L.A
Just anywhere away with you
I know we've got to get away
Someplace where no one knows
our name
We'll find the start to something new
Just take me anywhere,
take me anywhere
Anywhere away with you
Just take me anywhere,
take me anywhere
Anywhere away with you

Fun, little less fun
Little less, over, over,
over, over, me
Oh, fun, little less fun
Little less, over, over,
over, over, me

Truth comes out when we're blacking out
Looking for connection
in a crowd of empty faces,
empty faces
Your secrets are the only
thing I'm craving now
The good, and the bad, let me in
'Cause I can take it, I can take it

You're painting me a dream that I
Wanna belong, yeah, wanna belong, yeah

Over the h

In [86]:
print(df_en['Lyric'].iloc[1427])

I can't survive
Living without you baby
Deep in the night
I feel so cold inside

We used to say
"We'll be as one forever"
No matter where you are
I'll be loving you

So far away
You're so far away from me
I can't live without you baby
So far away
You're so far away from me


In [110]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(lyrics):
    sentiment = analyzer.polarity_scores(lyrics)
    return sentiment['neg']

sentiment1 = get_sentiment_score(lemmatized_corpus[496])
sentiment2 = get_sentiment_score(lemmatized_corpus[274])

tfidf_vectorizer = TfidfVectorizer()
lyrics_matrix = tfidf_vectorizer.fit_transform([lemmatized_corpus[496], lemmatized_corpus[274]])
lyrics_similarity = cosine_similarity(lyrics_matrix)

print("Sentiment Score for Song 1:", sentiment1)
print("Sentiment Score for Song 2:", sentiment2)
print("Cosine Similarity between Lyrics:", lyrics_similarity[0][1])

Sentiment Score for Song 1: 0.171
Sentiment Score for Song 2: 0.378
Cosine Similarity between Lyrics: 0.17731845777226657


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\trevo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
## Preprocessing
# Tokenize
# Lowercase
# punctuation removal
# number removal
# Stop word removal

## Additional Preprocessing
# Lemmatization and or stemming
# Analysis word frequency determine if we need to remove high frequency low value words
# Stop word removal

## Data Check
# Explore word stats
# Everything is clean

## NLP Processing
# Sentiment Analysis
# LDA
# POS taging/Similarity

## Front End
# Django or Flask additions

Test